In [ ]:
# imports
import requests
import pandas as pd


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
def get_foursquare_data(latitude, longitude, foursquare_api_key, radius=1000, limit=50):
    url = 'https://api.foursquare.com/v3/places/search'
    headers = {
        'Accept': 'application/json',
        'Authorization': foursquare_api_key
    }
    params = {
        'll': f'{latitude},{longitude}',
        'radius': radius,
        'limit': limit
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        raise Exception('Error with Foursquare API call')
    return response.json()


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
results = []
for index, row in df_stations.iterrows():
    if index > 240:
        break
    latitude = row['latitude']
    longitude = row['longitude']
    try:
        data = get_foursquare_data(latitude, longitude, 'FOURSQUARE_API_KEY')['results']
        for item in data:
            item_data = {key: item.get(key) for key in item}
            item_data.update({'Latitude': latitude, 'Longitude': longitude})
            results.append(item_data)
    except Exception as e:
        print(f'Failed to fetch data for station at index {index}: {e}')

df_results = pd.DataFrame(results)


Put your parsed results into a DataFrame

In [ ]:
df_results = pd.DataFrame(results)
df_results


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests
import pandas as pd

yelp_api_key = 'YELPKEY'

def get_yelp_data(latitude, longitude, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': 1000,
        'limit': 50
        # Add other parameters as needed
    }
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        raise Exception('Error with Yelp API call')
    return response.json()

results_yelp = []

for index, row in df_stations.iterrows():
    if index > 240:
        break

    latitude = row['latitude']
    longitude = row['longitude']

    try:
        data = get_yelp_data(latitude, longitude, yelp_api_key)['businesses']

        for item in data:
            results_yelp.append({
                'Latitude': latitude,
                'Longitude': longitude,
                'Yelp Name': item.get('name'),
                'Distance': item.get('distance'),
                'Rating': item.get('rating'),
                'Review Count': item.get('review_count'),
                'Categories': [category['title'] for category in item.get('categories')],
                'Price': item.get('price')
            })
    except Exception as e:
        print(f'Failed to fetch Yelp data for station at index {index}: {e}')

df_results_yelp = pd.DataFrame(results_yelp)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating